In [2]:
!pip install control

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 398 kB 13.9 MB/s 
  Created wheel for control: filename=control-0.9.2-py2.py3-none-any.whl size=403205 sha256=cb5972fdc010a89aff69ef7f85edb4c2c439f0bdb38fd3c5afa0b6eb1455afb3
  Stored in directory: /root/.cache/pip/wheels/48/ef/c2/929bb5c59a1328df00a0561d0e68cd7c8537f33f5ce0ce741b
Successfully built control


In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
 
# Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'Semesterproject 1/Code/KalmanNet_control/'
assert FOLDERNAME is not None, "[!] Enter the foldername."
 
# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
import torch

from systems import LinearSystem
from system_model_lqr import SystemModelLQR
from system_knet_linear import KalmanNetNN
from system_pipeline_loop import Pipeline_KF_loop

from system_model_finder import ModelFinder
from Linear_KF import KalmanFilter
from Plot import Plot

from datetime import datetime
import matplotlib.pyplot as plt
import os
from math import log10, cos, sin, pi

Running on the CPU


In [3]:
#############
### Model ###
#############
dtype_ = torch.float32
q2 = 1  # process noise variance
r2 = 1   # observation noise variance
T = 100    # trajectory length / time horizon
q2_dB = 10*log10(q2)
r2_dB = 10*log10(r2)

# State and observation dynamics
F = torch.tensor([[1, 1], [0,  1]], dtype=dtype_)
G = torch.tensor([[0],[1]], dtype=dtype_)
H = torch.tensor([[1, 0], [0, 1]], dtype=dtype_)

m = F.size()[0] # state dim
p = G.size()[1] # input dim
n = H.size()[0] # output dim

# Process and measurement noise
Q = q2 * torch.eye(m, dtype=dtype_)
R = r2 *  torch.eye(n, dtype=dtype_)  

# Initial state
m1_0 = x0 = torch.tensor([10, 0], dtype=dtype_)
xT = torch.tensor([0, 0], dtype=dtype_)
m2_0 = 0.*torch.eye(m) # covariance of initial state

# LQR stuff: cost matrices
scale = 1
qT = 1
qx = 1
qu = 0.1
QT = qT * torch.eye(m, dtype=dtype_) * scale
Qx = qx * torch.eye(m, dtype=dtype_) * scale
Qu = qu * torch.eye(p, dtype=dtype_) * scale

# Prior covariances for KalmanNet architecture 2
prior_Q = 1 * torch.eye(m, dtype=dtype_)
prior_Sigma = 1 * torch.eye(m, dtype=dtype_)
prior_S = 1 * torch.eye(n, dtype=dtype_)

###################
### True System ###
###################
a_deg = 0
a = a_deg / 180 * pi
Rot = torch.tensor([[cos(a), -sin(a)], [sin(a),  cos(a)]])
trueF = torch.matmul(Rot, F)
trueG = G
trueH = H

# Create system
true_sys = LinearSystem(trueF, trueG, trueH, Q, R)

# Create model
sys_model = SystemModelLQR(F, G, q2, H, r2, T, T, true_sys, prior_Q, prior_Sigma, prior_S)
sys_model.InitCostMatrices(QT, Qx, Qu)
sys_model.InitSequence(m1_0, m2_0)


In [4]:
#################
### Set Paths ###
#################

# TODO: set correct paths for desired experiment

drive_knet = '/content/drive/MyDrive/Semesterproject 1/Code/KalmanNet_control/'
data_path = drive_knet + 'final_experiments/for_presentation/'
data = f'N_10000_2000_2000_T_{T}_q2_{int(q2_dB)}dB_r2_{int(r2_dB)}dB.pt'
noise_data = data_path + 'noise_' + data

# Where to save the new model
models_folder = data_path + 'Test/'
model_name =  f'T_{T}_noise_{int(r2_dB)}dB_QT_{qT:.0e}_Qx_{qx:.0e}_Qu_{qu:.0e}_reg_1e-3_lr_5e-4_alpha_0_beta_1.pt'

# Make sure the folder already exists
error_message = f"The following directory does not exist:\n{models_folder}"
assert os.path.isdir(models_folder), error_message

# Remember to create a new name
warning = f"Model {model_name} already exists. If you continue, it will be overwritten."
assert not os.path.isfile(models_folder + 'model_' + model_name), warning

AssertionError: ignored

In [5]:
#################
### Load data ###
#################
training_noise, validation_noise, test_noise = torch.load(noise_data)

N_train = 1000
N_val = 100
N_test = 100

training_noise = [D[:N_train] for D in training_noise]

validation_noise = [D[:N_val] for D in validation_noise]

test_noise = [D[:N_test] for D in test_noise]

# Create initial and target states. The code gives the possibility to have 
# random states. However, so far it was not necessary.
X0_train = torch.ones(N_train, m) * x0
X0_val = torch.ones(N_val, m) * x0
X0_test = torch.ones(N_test, m) * x0

XT_train = torch.zeros_like(X0_train)
XT_val = torch.zeros_like(X0_val)
XT_test = torch.zeros_like(X0_test)


In [8]:
########################
### Optimal LQR Cost ###
########################
# To get an idea what the training error should converge to

my_LQR_cost = sys_model.EstimateLQRCost(X0_test, XT_test, N_test, test_noise, model=True)
my_dB_LQR_cost = 10*torch.log10(my_LQR_cost)
print(f'Estimated LQR cost with full state knowledge, model: {my_dB_LQR_cost} dB')

# my_LQR_cost_true_sys = sys_model.EstimateLQRCost(X0_test, XT_test, N_test, test_noise, model=False)
# my_dB_LQR_cost_true_sys = 10*torch.log10(my_LQR_cost_true_sys)
# print(f'Estimated LQR cost with full state knowledge, true system: {my_dB_LQR_cost_true_sys} dB')

my_LQG_cost = sys_model.EstimateLQGCost(X0_test, XT_test, N_test, test_noise, model=True)
my_dB_LQG_cost = 10*torch.log10(my_LQG_cost)
print(f'Estimated LQG cost with KF state estimation, model: {my_dB_LQG_cost} dB')

# my_LQG_cost_true_sys = sys_model.EstimateLQGCost(X0_test, XT_test, N_test, test_noise, model=False)
# my_dB_LQG_cost_true_sys = 10*torch.log10(my_LQG_cost_true_sys)
# print(f'Estimated LQR cost with KF state estimation, true system: {my_dB_LQG_cost_true_sys} dB')

Estimated LQR cost with full state knowledge, model: 9.1678466796875 dB
Estimated LQG cost with KF state estimation, model: 11.30194091796875 dB


In [ ]:
today = datetime.today()
now = datetime.now()
strToday = today.strftime("%m.%d.%y")
strNow = now.strftime("%H:%M:%S")
strTime = strToday + "_" + strNow

##################
###  KalmanNet ###
##################
inputs = (X0_train, X0_val)
targets = (XT_train, XT_val)

KNet_Pipeline = Pipeline_KF_loop(strTime, models_folder, model_name)
KNet_Pipeline.LQR_cost = my_dB_LQR_cost
KNet_Pipeline.LQG_cost = my_dB_LQG_cost
# KNet_Pipeline.LQR_cost_true_system = my_dB_LQR_cost_true_sys
# KNet_Pipeline.LQG_cost_true_system = my_dB_LQG_cost_true_sys
KNet_Pipeline.setssModel(sys_model)
KNet_model = KalmanNetNN()
KNet_model.Build(sys_model)
KNet_Pipeline.setModel(KNet_model)
KNet_Pipeline.setTrainingParams(n_Epochs=100, n_Batch=16, learningRate=0.5e-3, weightDecay=1e-3, alpha=0.0, beta=1.0)

################
### Training ###
################
KNet_Pipeline.NNTrain_state_access(inputs, targets, training_noise, validation_noise, n_val=16, num_restarts=2)
LQG_loss_summary, MSE_loss_total_summary, MSE_loss_position_summary = KNet_Pipeline.NNTest(X0_test, XT_test, test_noise)
KNet_Pipeline.save()

In [10]:
KNet_Pipeline.save()

In [ ]:
####################
### Plot results ###
####################
p = Plot(KNet_Pipeline)
p.plot_lqr_and_mse(-1.9, figSize=(25,20), lineWidth=3, ylim2=[8.5,25], ylim3=[-2.5,16], fontSize=26, saveName='model_'+model_name[:-3]+'_plots')

In [ ]:
#########################
### Continue Training ###
#########################
# Can continue training by loading the pipeline and starting the training again. 
# Make sure to set the KalmanNet as the best model from the previous training.

# Load existing pipeline
KNet_continue = torch.load(models_folder + 'pipeline_' + model_name)

# Load best model and set it as the starting model for the pipeline
best_model = torch.load(KNet_continue.modelFileName)
KNet_continue.model = best_model

# Maybe change name to not overwrite old one
new = '_update.pt'
KNet_continue.modelFileName = KNet_continue.modelFileName.replace('.pt',new)
KNet_continue.PipelineName = KNet_continue.PipelineName.replace('.pt',new)

# KNet_continue.N_Epochs = 100

# Set (potentially) new training parameters
KNet_continue.setTrainingParams(n_Epochs=100, n_Batch=16, learningRate=0.5e-3, weightDecay=1e-3, lr_decay=0.1, alpha=1.0, beta=1.0)

inputs = (X0_train, X0_val)
targets = (XT_train, XT_val)

# Make sure to have the same inputs and targets as in the previous training
KNet_continue.NNTrain_state_access(inputs, targets, training_noise, validation_noise, n_val=16, threshold=my_dB_LQR_cost, T=50)
LQG_loss_summary, LQR_loss_summary, MSE_loss_total_summary, MSE_loss_position_summary = KNet_continue.NNTest(X0_test, XT_test, test_noise)
KNet_continue.save()